# Test notebook to exercise the alert consumers

Important: change this to the correct import!

In [1]:
import sys
sys.path.append('../../lasair/lasair_api')

Change this to the endpoint, kafka server, and topic you want to test

In [2]:
kafka_server = 'kafka.lsst.ac.uk:9092'
topic        = "lasair_2Zooniverse"

Fetch your API TOKEN from your Lasair account, top right under "profile"

In [3]:
import json
from lasair import LasairError, lasair_consumer

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Normal Consumer

In [4]:
lsst=False
if lsst:
    sys.path.append('API_lsst')
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
else:
    sys.path.append('API_ztf')
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"

In [5]:
group_id     = 'test120'
consumer = lasair_consumer(
    kafka_server,    # kafka.lsst.ac.uk:9092
    group_id,        # any string
    topic            # topic name for your filter
)
import json
n = 0
while n < 10:
    msg = consumer.poll(timeout=20)
    if msg is None:
        break
    if msg.error():
        print(str(msg.error()))
        break
    jmsg = json.loads(msg.value())
    print(json.dumps(jmsg, indent=2))
    n += 1
print('No more messages available')

{
  "objectId": "ZTF24aatyini",
  "ramean": 343.0371050260869,
  "decmean": 3.7485526782608694,
  "mjdmin": 60518.34542819997,
  "mjdmax": 60544.33987270016,
  "magrmin": 18.0407,
  "rmag": 18.9873,
  "classification": "SN",
  "score": "Not Near PS1 star",
  "UTC": "2024-08-22 10:40:34"
}
{
  "objectId": "ZTF24aavkhwi",
  "ramean": 338.6713802538461,
  "decmean": 1.9816909615384615,
  "mjdmin": 60516.332916699816,
  "mjdmax": 60544.330613399856,
  "magrmin": 18.0456,
  "rmag": 18.8552,
  "classification": "NT",
  "score": "Not Near PS1 star",
  "UTC": "2024-08-22 10:46:12"
}
{
  "objectId": "ZTF24aawakwl",
  "ramean": 349.6628965045455,
  "decmean": 15.959210231818183,
  "mjdmin": 60514.43374999985,
  "mjdmax": 60544.37873840006,
  "magrmin": 17.1134,
  "rmag": 18.0223,
  "classification": "SN",
  "score": "Not Near PS1 star",
  "UTC": "2024-08-22 11:00:22"
}
{
  "objectId": "ZTF24aatsiox",
  "ramean": 338.42658783125,
  "decmean": -9.5207804625,
  "mjdmin": 60516.39795139991,
  "mjdma

## Rate Limited Consumer

In [6]:
from lasair import RL_consumer
group_id     = 'test121'
sqllite_filename = './%s.sql' % topic
rc = RL_consumer.lasair_RL_consumer(
    kafka_server,       # kafka.lsst.ac.uk:9092
    group_id,           # any string
    topic               # topic name for your filter
    sqllite_filename,   # past history
    interval=1,         # how many days before release  
    verbose=False
)

In [7]:
while 1:
    msg = rc.poll()
    if msg is None:
        break;
    objectId = json.loads(msg.value())['objectId']
    print('---> object %s found at %s' % (objectId, RL_consumer.now_human()))

---> object ZTF24aatyini found at 2024-09-04 14:47:50.681919
---> object ZTF24aavkhwi found at 2024-09-04 14:47:50.687110
---> object ZTF24aawakwl found at 2024-09-04 14:47:50.689546
---> object ZTF24aatsiox found at 2024-09-04 14:47:50.693099
---> object ZTF24aaybugo found at 2024-09-04 14:47:50.696383
---> object ZTF24aaxusdr found at 2024-09-04 14:47:50.699218
---> object ZTF24aaksujl found at 2024-09-04 14:47:50.702599
---> object ZTF24aaomrwv found at 2024-09-04 14:47:50.705408
---> object ZTF24aaudlat found at 2024-09-04 14:47:50.707885
---> object ZTF23aaensha found at 2024-09-04 14:47:50.709756
---> object ZTF24aaydpre found at 2024-09-04 14:47:50.712452
---> object ZTF24aavzkuf found at 2024-09-04 14:47:50.716167
---> object ZTF23aamksid found at 2024-09-04 14:47:50.717782
---> object ZTF24aatsmxc found at 2024-09-04 14:47:50.719241
---> object ZTF24aaerzgz found at 2024-09-04 14:47:50.721088
---> object ZTF24aaaeljz found at 2024-09-04 14:47:50.722471
---> object ZTF24aaydzzk

In [8]:
rc.printall()

lasair_2Zooniverse
At 2024-09-04 14:48:02.207599
ZTF23aavdauu accepted 0.000 days ago, rejected 0 times
ZTF24aanvvqn accepted 0.000 days ago, rejected 0 times
ZTF24aapexhw accepted 0.000 days ago, rejected 0 times
ZTF24aaymskk accepted 0.000 days ago, rejected 0 times
ZTF24aawxsjw accepted 0.000 days ago, rejected 0 times
ZTF24aawyyma accepted 0.000 days ago, rejected 0 times
ZTF24abbiqob accepted 0.000 days ago, rejected 1 times
ZTF24aazszfd accepted 0.000 days ago, rejected 1 times
ZTF24abatbnd accepted 0.000 days ago, rejected 0 times
ZTF24aafraez accepted 0.000 days ago, rejected 0 times
ZTF24aamynfp accepted 0.000 days ago, rejected 2 times
ZTF24abbinrr accepted 0.000 days ago, rejected 0 times
ZTF24abbenwl accepted 0.000 days ago, rejected 1 times
ZTF24abamnfx accepted 0.000 days ago, rejected 0 times
ZTF24aaxnbot accepted 0.000 days ago, rejected 0 times
ZTF24abbbjab accepted 0.000 days ago, rejected 1 times
ZTF24aathuwg accepted 0.000 days ago, rejected 0 times
ZTF24abamchy acc